In [2]:
import pandas as pd
import numpy as np
from load_data import IHDP_Dataset
from dowhy import CausalModel

import torch
from econml.dml import CausalForestDML
from sklearn.linear_model import LassoCV, LinearRegression
from sklearn.model_selection import train_test_split

In [3]:
# データの読み込み
ihdp_train_D = IHDP_Dataset(train=True, mono=True)
ihdp_test_D = train_D = IHDP_Dataset(train=False, mono=True)
ate_true = torch.mean(ihdp_test_D.mu1 - ihdp_test_D.mu0)

### Causal ForestによるITE, ATEの推定

In [8]:
# causal forestの訓練
covariates = [f'x{i+1}' for i in range(25)]
treatment = 'treatment'
outcome = 'yf'
label = covariates + [treatment, outcome]
df = pd.DataFrame(np.c_[ihdp_train_D.x, ihdp_train_D.t, ihdp_train_D.yf], columns=label)
model = CausalModel(data=df, treatment=treatment, outcome=outcome, common_causes=covariates, instruments=None, effect_modifiers=None)
# model.view_model()

train_data, test_data = train_test_split(df, test_size=0.2)

X_tr, Z_tr, Y_tr = train_data[covariates], train_data[treatment], train_data[outcome]
causal_forest = CausalForestDML(
    criterion='het',
    n_estimators=10000,
    min_samples_leaf=10,
    max_depth=None,
    max_samples=0.5,  # Q?: なぞ
    discrete_treatment=False,  # Q?: なぞ
    honest=True,  # Q?: Causal Treeを使うのですべてhonestでは？なぜわざわざ指定するオプションがある?
    inference=True,  # Q?: 推論しないなら何すんねん
    cv=10,
    model_t=LassoCV(),
    model_y=LassoCV()
)
causal_forest.fit(Y_tr, Z_tr, X=X_tr, W=None)  # Q?: Wがなぞ

In [9]:
np.mean(causal_forest.effect(X=ihdp_test_D.x, T1=torch.ones(ihdp_test_D.N), T0=torch.zeros(ihdp_test_D.N)))

In [10]:
# Within Sample, Out of Sampleの評価で共通する手順

def loss_pehe(X, mu1, mu0):
    effect_hat = torch.from_numpy(causal_forest.effect(X=X, T1=torch.ones(X.shape[0]), T0=torch.zeros(X.shape[0])))  # 効果の推定量
    effect_true = mu1 - mu0  # 真の効果
    return torch.sqrt(torch.mean((effect_hat - effect_true)**2))

def loss_ate(X, mu1, mu0):
    effect_hat = torch.from_numpy(causal_forest.effect(X=X, T1=torch.ones(X.shape[0]), T0=torch.zeros(X.shape[0])))  # 効果の推定量
    effect_true = mu1 - mu0  # 真の効果
    ate_hat = torch.mean(effect_hat)  # ATEの推定量
    ate_true = torch.mean(effect_true)  # 真のATE
    return torch.abs(ate_hat - ate_true)

def evaluation(D, fname):
    # \epsilon_{PEHE}を算出
    pehe = loss_pehe(D.x, D.mu1, D.mu0)

    # \epsilon_{ATE}を算出
    ate_error = loss_ate(D.x, D.mu1, D.mu0)

    # 結果の表示
    print('pehe = ', pehe.item())
    print('error of ate =', ate_error.item())

    # 結果の保存
    torch.save({
        'X_test': D.x,
        'pehe': pehe,
        'ate_error': ate_error,
    }, fname)

In [13]:
# Within Sample
evaluation(ihdp_train_D, 'results/causal_forest_WS.pt')

pehe =  0.5815477004277092
error of ate = 0.15041796288156872


In [14]:
# Out of Sample
evaluation(ihdp_test_D, 'results/causal_forest_OoS.pt')

pehe =  0.6078326950202498
error of ate = 0.20653365771842802


### 疑問
* `causal_forest.effect_inference(X=X_test, T1=Z_test)`をした時に帰ってくる`InferenceResults`オブジェクトが謎
    * しかし、今は必要ないのでまた気になったら調べる。
* model_y, model_tにLassoを用いているが, CausalTreeでそんなんあったけ？
    * CausalTreeの内容がまだわかっていないようだな。理解し直しなさい。
    * ITE予測の研究をしないならいらないかもしれないが、教養として
* `causal_forest.marginal~`とかあるけど、何に関する周辺かをしてるのか, なぜしてるのかを理解してない。